In [ ]:

from pathlib import Path
import shutil
import zipfile

# Kaggle の入力データセット slug（username/slug の slug 部分）
DATASET_SLUG = 'gns-codes'
DATASET_ROOT = Path(f'/kaggle/input/{DATASET_SLUG}')
WORK_ROOT = Path('/kaggle/working')
repo_dir = WORK_ROOT / 'code'

code_dir = DATASET_ROOT / 'code'
code_zip = DATASET_ROOT / 'code.zip'

# 展開済み code/ を優先し、無ければ code.zip を展開
if code_dir.exists():
    src = code_dir
elif code_zip.exists():
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    with zipfile.ZipFile(code_zip) as zf:
        zf.extractall(repo_dir)
    src = repo_dir
else:
    raise FileNotFoundError(f"/kaggle/input/{DATASET_SLUG} に code も code.zip もありません。Add Data で {DATASET_SLUG} を追加してください。")

# フラット化（zip解凍で1階層挟まった場合）
if src != repo_dir:
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    shutil.copytree(src, repo_dir)
if repo_dir.exists():
    children = list(repo_dir.iterdir())
    if len(children) == 1 and children[0].is_dir():
        inner = children[0]
        for p in inner.iterdir():
            p.rename(repo_dir / p.name)
        inner.rmdir()

%cd $repo_dir


In [ ]:

# PyG の radius_graph で必要になる torch-cluster だけインストール
import torch

torch_ver = torch.__version__.split('+')[0]
cuda_ver = torch.version.cuda
cuda_tag = 'cpu' if cuda_ver is None else 'cu' + cuda_ver.replace('.', '')
url = f"https://data.pyg.org/whl/torch-{torch_ver}+{cuda_tag}.html"
print('PyG wheels:', url)

!pip -q install torch-cluster -f {url}
!pip -q install torch_geometric


In [ ]:
from pathlib import Path
import re
import yaml

%cd /kaggle/working/code

import sys
sys.path.append(str(Path.cwd() / 'src'))

from train_config import Config

REPO = Path('/kaggle/working/code')
cfg_src = REPO / 'config_rollout.yaml'

# 推論に使うデータセット（Kaggle Dataset として追加してください）
dataset_root = "/kaggle/input/dam-break-left"

# 出力設定
output_root = REPO / 'rollouts'
output_filename = 'inference_result'
viz_format = 'html'  # html|mp4|gif

with cfg_src.open('r', encoding='utf-8') as f:
    cfg_dict = yaml.safe_load(f)

cfg_dict['method'] = 'gns'
cfg_dict['rollout_inference_max_examples'] = 1
cfg_dict.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

# モデルはコードに同梱された models/ から取得
model_dir = REPO / 'models'
cfg_dict['model_path'] = str(model_dir)

resolved_model_file = None
if model_dir.exists():
    expr = re.compile(r'model-(\d+)\.pt$')
    best = None
    for path in model_dir.rglob('model-*.pt'):
        m = expr.search(path.name)
        step = int(m.group(1)) if m else -1
        entry = (path.stat().st_mtime, step, path)
        if best is None or entry > best:
            best = entry
    if best:
        resolved_model_file = best[2]

if resolved_model_file is not None:
    cfg_dict['model_file'] = str(resolved_model_file)
else:
    cfg_dict['model_file'] = 'latest'  # models/latest を想定

cfg_dict['output_path'] = str(output_root)
cfg_dict['output_filename'] = output_filename

cfg = Config(**cfg_dict)
print('model_file:', cfg.model_file)
print('output_path:', cfg.output_path)
print('output_filename:', cfg.output_filename)
print('dataset:', dataset_root)

In [ ]:
# import subprocess
# from pathlib import Path
# import torch

# from predictor import predict
# from train import _prepare_scenario
# from train_paths import _prepare_model_directory, _resolve_model_run_directory

# # 上のセルで用意した cfg をそのまま使用
# cfg.rank = 0
# cfg.world_size = 1
# cfg.local_rank = 0
# cfg.enable_ddp = False
# cfg = _prepare_scenario(cfg)
# _prepare_model_directory(cfg)
# _resolve_model_run_directory(cfg)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f'device: {device}')

# predict(cfg, device)

# # 可視化
# output_dir = Path(cfg.output_path) / cfg.method / cfg.output_filename
# pkl_path = output_dir / f"{cfg.output_filename}_ex0.pkl"
# viz_out = output_dir / f"{cfg.output_filename}.{viz_format}"

# viz_cmd = [
#     'python',
#     'visualize_rollout.py',
#     str(pkl_path),
#     '--format', viz_format,
#     '--output', str(viz_out),
# ]
# print(f"Visualizing: {' '.join(viz_cmd)}")
# subprocess.run(viz_cmd, check=True)

# print(f"\n✅ Complete! Output: {viz_out}")

In [ ]:
# 1-step 誤差を時間ステップごとに計算（tank_sloshing で評価）
import numpy as np
import torch

import data_loader
import reading_utils
from rollout_utils import rollout
from simulator_factory import _get_simulator
from train import _prepare_scenario
from train_config import INPUT_SEQUENCE_LENGTH
from train_paths import _prepare_model_directory, _resolve_model_run_directory, _resolve_model_path
from train_utils import _resolve_rollout_dataset_path

# cfg をそのまま再利用（未準備なら準備）
if getattr(cfg, 'active_scenario', None) is None:
    cfg = _prepare_scenario(cfg)
_prepare_model_directory(cfg)
_resolve_model_run_directory(cfg)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
metadata_key = cfg.active_scenario.rollout_metadata_split or 'rollout'
metadata = reading_utils.read_metadata(cfg.data_path, metadata_key)

simulator = _get_simulator(metadata, cfg.noise_std, cfg.noise_std, device, cfg)
model_path = _resolve_model_path(cfg)
print('モデルをロード:', model_path)
simulator.load(model_path)
simulator.to(device)
simulator.eval()

dataset_path = _resolve_rollout_dataset_path(cfg)
print('データセット:', dataset_path)
loader = data_loader.get_data_loader_by_trajectories(dataset_path)

max_eval_examples = None  # None で全軌道を評価
max_steps_cap = None      # 例: 50 なら 50 ステップまでに制限

rmse_time_series = []
dt = float(metadata.get('dt', 1.0))

with torch.no_grad():
    for ex_idx, features in enumerate(loader):
        if max_eval_examples is not None and ex_idx >= int(max_eval_examples):
            break

        positions = features[0].to(device)
        particle_type = features[1].to(device)
        if len(features) == 4:
            material_property = features[2].to(device)
            n_particles = torch.tensor([int(features[3])], dtype=torch.int32, device=device)
        else:
            material_property = None
            n_particles = torch.tensor([int(features[2])], dtype=torch.int32, device=device)

        nsteps = positions.shape[1] - INPUT_SEQUENCE_LENGTH
        if max_steps_cap is not None:
            nsteps = min(nsteps, int(max_steps_cap))

        _, loss = rollout(
            simulator,
            positions,
            particle_type,
            material_property,
            n_particles,
            nsteps,
            device,
            show_progress=False,
        )

        rmse_per_step = torch.sqrt(loss.mean(dim=(1, 2))).cpu().numpy()
        frame_axis = INPUT_SEQUENCE_LENGTH + np.arange(1, len(rmse_per_step) + 1)
        time_axis = dt * frame_axis
        rmse_time_series.append({
            'example': ex_idx,
            'frame': frame_axis,
            'time': time_axis,
            'rmse': rmse_per_step,
        })

print(f'評価完了: {len(rmse_time_series)} 件')



In [ ]:
# RMSE の時系列を可視化（各軌道＋平均）
import numpy as np
import matplotlib.pyplot as plt

if not rmse_time_series:
    raise RuntimeError('rmse_time_series が空です。前のセルを確認してください。')

max_len = max(len(entry['rmse']) for entry in rmse_time_series)
frame_grid = np.full((len(rmse_time_series), max_len), np.nan, dtype=float)
rmse_grid = np.full_like(frame_grid, np.nan)

plt.figure(figsize=(8, 5))
for i, entry in enumerate(rmse_time_series):
    l = len(entry['rmse'])
    frame_grid[i, :l] = entry['frame']
    rmse_grid[i, :l] = entry['rmse']
    plt.plot(entry['frame'], entry['rmse'], alpha=0.25, label=f"ex{entry['example']}")

mean_frame = np.nanmean(frame_grid, axis=0)
mean_rmse = np.nanmean(rmse_grid, axis=0)
plt.plot(mean_frame, mean_rmse, color='red', lw=2, label='mean')

plt.xlabel('frame index')
plt.ylabel('position RMSE (1-step)')
plt.title('dam-break: 1-step error vs. frame')
plt.grid(True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

